In [1]:
from __future__ import annotations

from datetime import  datetime
import ee
import geemap
from eelib import collections, eefuncs

In [2]:
TARGET_YEAR = '2018'

# set date ranges
doys = 135, 181, 182, 243, 244, 300 

def to_datetime(doys: tuple[str]):
    
    # julain days parse to YYYY MM dd
    def to_dt(doy: int) -> str:
        doy = datetime.strptime(f'{TARGET_YEAR}{doy}', '%Y%j')
        return doy.strftime("%Y-%m-%d")
    
    return tuple(map(to_dt, doys))
dates = to_datetime(doys)

spring, summer, fall = dates[:2], dates[2:4], dates[4:]
print(f'Spring:{spring}')
print(f'Summer:{summer}')
print(f'Fall:{fall}')
    


Spring:('2018-05-15', '2018-06-30')
Summer:('2018-07-01', '2018-08-31')
Fall:('2018-09-01', '2018-10-27')


In [3]:
viewport = collections.FeatureCollection.from_file(
    filename="../../000-data/intersection_64.shp",
    driver='ESRI Shapefile'
)

base = collections.S1Collection().\
    filterBounds(viewport).\
    filter('relativeOrbitNumber_start == 64').map(eefuncs.insert_groupid)

seasons = {
    'spring': {'collection': base.filterDate(*spring), 'date': spring},
    'summer': {'collection': base.filterDate(*summer),'date': summer},
    'fall': {'collection': base.filterDate(*fall),'date': fall}
}


In [4]:
# set the mid date for each season and difference the days
for season, obj in seasons.items():
    dates = (ee.Date(_) for _ in obj.get('date'))
    mid_point = eefuncs.get_mid_point(dates)
    
    # update the collection inplace
    trg =  eefuncs.days_from_mid(
        collection=obj.get('collection'),
        midPoint = mid_point
    )
    
    trg = trg.map(lambda x: x.set('season', season))
    
    print(f"{season}: Days from Mid")
    print(trg.aggregate_array('dfm').distinct().sort().getInfo())
    print(trg.size().getInfo())

    obj['collection'] = trg

spring: Days from Mid
[2, 9, 14, 21]
12
summer: Days from Mid
[4, 16, 28]
8
fall: Days from Mid
[15]
3


In [5]:
# query the collections based on days

# Full coverage = dfms == [2, 16, 9]

spring_images = seasons.get('spring').get('collection').filter('dfm == 2')
summer_images = seasons.get('summer').get('collection').filter('dfm == 16')
fall_images = seasons.get('fall').get('collection').filter('dfm == 15')


In [6]:
print(spring_images.aggregate_array('groupid').getInfo())
print(summer_images.aggregate_array('groupid').getInfo())
print(fall_images.aggregate_array('groupid').getInfo())

['64_-122.36', '64_-122.83', '64_-123.37']
['64_-122.36', '64_-122.83', '64_-123.37']
['64_-122.47', '64_-122.94', '64_-123.33']


In [10]:
col_list = [spring_images.toList(spring_images.size()),
            summer_images.toList(summer_images.size()),
            fall_images.toList(fall_images.size())]



EEException: ImageCollection.fromImages, argument 'images': Invalid type.
Expected type: List<Image<unknown bands>>.
Actual type: List<ImageCollection>.
Actual value: [<ImageCollection>, <ImageCollection>, <ImageCollection>]

In [7]:
Map = geemap.Map()

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
vis = {'bands': ['VV'], 'min': -25, 'max': 10}

Map.addLayer(spring_images, vis, 'Spring')
Map.addLayer(summer_images, vis, 'Summer')
Map.addLayer(fall_images, vis, 'Fall')
Map.addLayer(viewport, {}, 'View Port')
Map.addLayerControl()
